In [7]:
import numpy as np
import torch
import math
import pandas as pd
import copy 
from typing import Optional
from torch import Tensor
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import random
import warnings
warnings.filterwarnings("ignore")

#Botorch imports
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.optim import optimize_acqf
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import LogExpectedImprovement, ProbabilityOfImprovement, UpperConfidenceBound
from botorch.acquisition.monte_carlo import qLowerConfidenceBound
from gpytorch.kernels import MaternKernel, RBFKernel, ScaleKernel

In [8]:
class CustomGP(SingleTaskGP):
    def __init__(
        self,
        X,
        Y,
        kernel = None,
        ARD = True #Automatic Relevance Determination (ARD) for the kernel
    ):
        super().__init__(
            train_X=X,
            train_Y=Y,
        )
        if ARD == True:
            if kernel == "RBF" or kernel == None:
                pass #SingleTaskGP uses RBFKernel with ARD=True by default
            elif kernel == "Matern":
                self.covar_module = ScaleKernel(
                    base_kernel = MaternKernel(
                        nu=2.5,
                        ard_num_dims = X.shape[-1]
                    )
                )
        else:
            if kernel == "RBF" or kernel == None:
                self.covar_module = ScaleKernel(
                    base_kernel = RBFKernel(
                        ard_num_dims = None 
                    )
                )
            elif kernel == "Matern":
                self.covar_module = ScaleKernel(
                    base_kernel = MaternKernel(
                        nu=2.5,
                        ard_num_dims = None
                    )
                )
                
        self.to(X)
        
    
    
        
    


In [9]:
def prepare_data(dataset_path):
    raw_dataset = pd.read_csv(dataset_path + '_dataset.csv')
    feature_name = list(raw_dataset.columns)[:-1]
    objective_name = list(raw_dataset.columns)[-1]
    
    ds = copy.deepcopy(raw_dataset) 
    ds[objective_name] = -raw_dataset[objective_name].values #turn the problem into a minimization problem
    
    #drop duplicates and missing values
    ds_grouped = ds.groupby(feature_name)[objective_name].agg(lambda x: x.unique().mean())
    ds_grouped = (ds_grouped.to_frame()).reset_index()
    ds_grouped.dropna(inplace=True) 
    
    X_feature = ds_grouped[feature_name].values

    y = np.array(ds_grouped[objective_name].values)

    assert len(ds_grouped) == len(X_feature) == len(y)
    
    N = len(ds_grouped)
    
    return X_feature, y, N, feature_name, objective_name, ds_grouped
    

In [10]:
def pool_learning(X, y, model, N, acqui_fn, kernel, ARD, top_indices, n_initial, seed):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device: ", device)
    np.random.seed(seed)
    
    
    indices_to_learn = [i for i in range(len(X))] # indices that are still in the pool
    indices_learned= [] # indices that have been learned
    train_indices = random.sample(range(N), n_initial) # start with n_initial random points
    
    train_obj = torch.from_numpy(y[train_indices]).unsqueeze(-1).float() # objective values of these points
    train_x = torch.from_numpy(X[train_indices]).float() # input features of these points
    
    for i in train_indices:
        indices_to_learn.remove(i) # remove these points from the pool
    
    indices_learned.extend(train_indices) # add these points to learned points
    
    cnt = 0 #this counts the number of times the top indices are learned
    
    top_indices_cnt = []
    for index in indices_learned:
        if index in top_indices:
            cnt += 1
        top_indices_cnt.append(cnt)
                
    i = 0
    while len(indices_learned) < len(X):
        # initialize the model and fit it
        train_x = train_x.to(device)
        train_obj = train_obj.to(device)
        
        assert len(train_x) == len(train_obj)
        GP_model = model(train_x, train_obj, kernel, ARD).to(device)
        mll = ExactMarginalLogLikelihood(GP_model.likelihood, GP_model)
        fit_gpytorch_mll(mll)
        
        # apply the acquisition function
        best_value = train_obj.min()
        acqui = acqui_fn(model=GP_model, best_f = best_value, maximize=False).to(device)
        
        # Apply acquisition function to find the best index to learn next
        candidate_features = torch.from_numpy(X[indices_to_learn]).float().to(device)

        acq_values = np.array([acqui(i.unsqueeze(0)).detach().cpu().numpy() for i in candidate_features])
        
        print(i, "th iteration")
        i += 1
        # print(acq_values)
        next_index = indices_to_learn[np.argmax(acq_values)] # find the index with the highest acquisition value
        
        
        # add the new point to the training set
        train_indices.append(next_index)
        indices_to_learn.remove(next_index)
        indices_learned.append(next_index)
        
        # update the training set
        train_x = torch.from_numpy(X[train_indices]).float()
        train_obj = torch.from_numpy(y[train_indices]).unsqueeze(-1).float()
        
        #update record of top indices
        if next_index in top_indices:
            cnt += 1
        
        top_indices_cnt.append(cnt)
        if cnt == len(top_indices):
            break
        
        
        
    return indices_learned, top_indices_cnt
    

### Select hyperparameters

In [11]:
kernel = "RBF"  # Choose between "RBF" and "Matern"
ARD = False # Choose between True and False
dataset = './datasets/P3HT' # Chanage the name of the dataset in the end of the path


#Benchmarking Parameters
# number of ensembles. in the paper n_ensemble = 50.
X_feature, y, N, feature_name, objective_name, ds_grouped = prepare_data(dataset)
print(ds_grouped.head())
n_ensemble = 50
# number of initial experiments
n_initial = 10
# number of top candidates, currently using top 5% of total dataset size
n_top = int(math.ceil(len(y) * 0.05))
# the top candidates and their indicies
top_indices = list(ds_grouped.sort_values(objective_name).head(n_top).index)

# random seeds used to distinguish between different ensembles
# there are 300 of them, but only first n_ensemble are used
seed_list = [4295, 8508, 326, 3135, 1549, 2528, 1274, 6545, 5971, 6269, 2422, 4287, 9320, 4932, 951, 4304, 1745, 5956, 7620, 4545, 6003, 9885, 5548, 9477, 30, 8992, 7559, 5034, 9071, 6437, 3389, 9816, 8617, 3712, 3626, 1660, 3309, 2427, 9872, 938, 5156, 7409, 7672, 3411, 3559, 9966, 7331, 8273, 8484, 5127, 2260, 6054, 5205, 311, 6056, 9456, 928, 6424, 7438, 8701, 8634, 4002, 6634, 8102, 8503, 1540, 9254, 7972, 7737, 3410, 4052, 8640, 9659, 8093, 7076, 7268, 2046, 7492, 3103, 3034, 7874, 5438, 4297, 291, 5436, 9021, 3711, 7837, 9188, 2036, 8013, 6188, 3734, 187, 1438, 1061, 674, 777, 7231, 7096, 3360, 4278, 5817, 5514, 3442, 6805, 6750, 8548, 9751, 3526, 9969, 8979, 1526, 1551, 2058, 6325, 1237, 5917, 5821, 9946, 5049, 654, 7750, 5149, 3545, 9165, 2837, 5621, 6501, 595, 3181, 1747, 4405, 4480, 4282, 9262, 6219, 3960, 4999, 1495, 6007, 9642, 3902, 3133, 1085, 3278, 1104, 5939, 7153, 971, 8733, 3785, 9056, 2020, 7249, 5021, 3384, 8740, 4593, 7869, 9941, 8813, 3688, 8139, 6436, 3742, 5503, 1587, 4766, 9846, 9117, 7001, 4853, 9346, 4927, 8480, 5298, 4753, 1151, 9768, 5405, 6196, 5721, 3419, 8090, 8166, 7834, 1480, 1150, 9002, 1134, 2237, 3995, 2029, 5336, 7050, 6857, 8794, 1754, 1184, 3558, 658, 6804, 8750, 5088, 1136, 626, 8462, 5203, 3196, 979, 7419, 1162, 5451, 6492, 1562, 8145, 8937, 8764, 4174, 7639, 8902, 7003, 765, 1554, 6135, 1689, 9530, 1398, 2273, 7925, 5948, 1036, 868, 4617, 1203, 7680, 7, 93, 3128, 5694, 6979, 7136, 8084, 5770, 9301, 1599, 737, 7018, 3774, 9843, 2296, 2287, 9875, 2349, 2469, 8941, 4973, 3798, 54, 2938, 4665, 3942, 3951, 9400, 3094, 2248, 3376, 1926, 5180, 1773, 3681, 1808, 350, 6669, 826, 539, 5313, 6193, 5752, 9370, 2782, 8399, 4881, 3166, 4906, 5829, 4827, 29, 6899, 9012, 6986, 4175, 1035, 8320, 7802, 3777, 6340, 7798, 7705]


   P3HT content (%)  D1 content (%)  D2 content (%)  D6 content (%)  \
0             15.00            0.00            0.00           85.00   
1             16.15            0.46           25.38           51.42   
2             16.88            1.25           24.96           49.58   
3             17.71            0.98           25.34           50.30   
4             18.67            0.27           47.09           24.03   

   D8 content (%)  Conductivity (measured) (S/cm)  
0            0.00                         -45.970  
1            6.55                         -16.905  
2            7.32                         -20.030  
3            5.68                         -14.780  
4            9.99                         -14.080  


In [12]:
results = []

for i in tqdm(range(n_ensemble)):
    print(f"Running ensemble {i+1} with {kernel} kernel")
    model = CustomGP
    acqui_fn = LogExpectedImprovement
    indices, top_indices_cnt = pool_learning(
        X=X_feature,
        y=y,
        model=model,
        kernel = kernel,
        ARD = ARD,
        acqui_fn=acqui_fn,
        top_indices=top_indices,
        n_initial=n_initial,
        N = N,
        seed=seed_list[i]
    )
    results.append(top_indices_cnt)

  0%|          | 0/50 [00:00<?, ?it/s]

Running ensemble 1 with RBF kernel
Using device:  cuda
0 th iteration
1 th iteration
2 th iteration
3 th iteration
4 th iteration
5 th iteration
6 th iteration
7 th iteration
8 th iteration
9 th iteration
10 th iteration
11 th iteration
12 th iteration
13 th iteration
14 th iteration
15 th iteration
16 th iteration
17 th iteration
18 th iteration
19 th iteration
20 th iteration
21 th iteration
22 th iteration
23 th iteration
24 th iteration
25 th iteration
26 th iteration
27 th iteration
28 th iteration
29 th iteration
30 th iteration
31 th iteration
32 th iteration
33 th iteration
34 th iteration
35 th iteration
36 th iteration
37 th iteration
38 th iteration
39 th iteration
40 th iteration
41 th iteration
42 th iteration
43 th iteration
44 th iteration
45 th iteration
46 th iteration
47 th iteration
48 th iteration
49 th iteration
50 th iteration
51 th iteration
52 th iteration
53 th iteration
54 th iteration
55 th iteration
56 th iteration
57 th iteration
58 th iteration
59 th itera

  2%|▏         | 1/50 [00:43<35:51, 43.91s/it]

130 th iteration
Running ensemble 2 with RBF kernel
Using device:  cuda
0 th iteration
1 th iteration
2 th iteration
3 th iteration
4 th iteration
5 th iteration
6 th iteration
7 th iteration
8 th iteration
9 th iteration
10 th iteration
11 th iteration
12 th iteration
13 th iteration
14 th iteration
15 th iteration
16 th iteration
17 th iteration
18 th iteration
19 th iteration
20 th iteration
21 th iteration
22 th iteration
23 th iteration
24 th iteration
25 th iteration
26 th iteration
27 th iteration
28 th iteration
29 th iteration
30 th iteration
31 th iteration
32 th iteration
33 th iteration
34 th iteration
35 th iteration
36 th iteration
37 th iteration
38 th iteration
39 th iteration
40 th iteration
41 th iteration
42 th iteration
43 th iteration
44 th iteration
45 th iteration
46 th iteration
47 th iteration
48 th iteration
49 th iteration
50 th iteration
51 th iteration
52 th iteration
53 th iteration
54 th iteration
55 th iteration
56 th iteration
57 th iteration
58 th iter

  4%|▍         | 2/50 [01:24<33:25, 41.77s/it]

...

138 th iteration
Running ensemble 49 with RBF kernel
Using device:  cuda
0 th iteration
1 th iteration
2 th iteration
3 th iteration
4 th iteration
5 th iteration
6 th iteration
7 th iteration
8 th iteration
9 th iteration
10 th iteration
11 th iteration
12 th iteration
13 th iteration
14 th iteration
15 th iteration
16 th iteration
17 th iteration
18 th iteration
19 th iteration
20 th iteration
21 th iteration
22 th iteration
23 th iteration
24 th iteration
25 th iteration
26 th iteration
27 th iteration
28 th iteration
29 th iteration
30 th iteration
31 th iteration
32 th iteration
33 th iteration
34 th iteration
35 th iteration
36 th iteration
37 th iteration
38 th iteration
39 th iteration
40 th iteration
41 th iteration
42 th iteration
43 th iteration
44 th iteration
45 th iteration
46 th iteration
47 th iteration
48 th iteration
49 th iteration
50 th iteration
51 th iteration
52 th iteration
53 th iteration
54 th iteration
55 th iteration
56 th iteration
57 th iteration
58 th ite

 98%|█████████▊| 49/50 [36:21<00:44, 44.98s/it]

144 th iteration
Running ensemble 50 with RBF kernel
Using device:  cuda
0 th iteration
1 th iteration
2 th iteration
3 th iteration
4 th iteration
5 th iteration
6 th iteration
7 th iteration
8 th iteration
9 th iteration
10 th iteration
11 th iteration
12 th iteration
13 th iteration
14 th iteration
15 th iteration
16 th iteration
17 th iteration
18 th iteration
19 th iteration
20 th iteration
21 th iteration
22 th iteration
23 th iteration
24 th iteration
25 th iteration
26 th iteration
27 th iteration
28 th iteration
29 th iteration
30 th iteration
31 th iteration
32 th iteration
33 th iteration
34 th iteration
35 th iteration
36 th iteration
37 th iteration
38 th iteration
39 th iteration
40 th iteration
41 th iteration
42 th iteration
43 th iteration
44 th iteration
45 th iteration
46 th iteration
47 th iteration
48 th iteration
49 th iteration
50 th iteration
51 th iteration
52 th iteration
53 th iteration
54 th iteration
55 th iteration
56 th iteration
57 th iteration
58 th ite

100%|██████████| 50/50 [37:09<00:00, 44.59s/it]

137 th iteration


In [13]:
saved = np.array(results, dtype=object)
print(saved)
np.save('results_P3HT_RBF_NOARD_EI_Init10.npy', saved)

[list([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9])
 list([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 9])
 list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,